# Python字典示例：用户验证
## 用户输入密码加密，循环验证用户输入密码合法性

In [3]:
import hashlib  # 数据加密与完整性交易的核心
import hmac  # 适用于API签名，安全通信等场景
import os  # 跨平台操作

# 用户数据库（新增注册功能）
users = {}


def hash_password(password: str, salt: str) -> str:
    """使用PBKDF2_HMAC算法对密码进行加盐哈希"""
    # 参数说明：
    # hashlib.sha256 - 使用SHA-256哈希算法
    # password.encode() - 将密码字符串转为bytes
    # salt.encode() - 将盐字符串转为bytes
    # 100000 - 迭代次数（增加计算成本，防止暴力破解）
    # dklen=32 - 输出哈希长度（32字节=256位）
    hashed = hashlib.pbkdf2_hmac(
        'sha256',
        password.encode('utf-8'),
        salt.encode('utf-8'),
        100000,
        dklen=32
    )
    return hashed.hex()  # 转换为十六进制字符串存储


def register_user():
    username = input("注册用户名：")
    while username in users:
        print("用户名已存在")
        username = input("注册用户名：")

    while True:
        password = input("设置密码(需包含大小写字母、数字且>=8位)：")
        if not re.match(r'^(?=.*[A-Z])(?=.*[a-z])(?=.*\d).{8,}$', password):
            # re.match：Python的re模块中的函数，用于检查字符串是否匹配给定的正则表达式。
            # ^ 和 $：分别表示字符串的开始和结束，确保整个密码字符串都符合规则。
            # (?=.*[A-Z])：正向预查（positive lookahead），确保字符串中至少有一个大写字母（[A-Z]）。
            # (?=.*[a-z])：正向预查，确保字符串中至少有一个小写字母（[a-z]）。
            # (?=.*\d)：正向预查，确保字符串中至少有一个数字（\d）。
            # .{8,}：匹配任意字符（.）至少8次（{8,}），确保密码长度至少为8。
            # if not re.match(...)：如果密码不符合正则表达式的要求，则条件成立（进入if块）。
            print("密码需要包含至少1个大写字母、1个小写字母和1个数字，且长度至少8位")

            continue
        break

    # 生成随机盐（16个字节=128位）
    # 16*8-128位密码学安全随机数，并转换为16*2=32位十六进制字符串（每四位对应一个字符）
    salt = os.urandom(16).hex()
    # 存储加盐哈希后的密码
    users[username] = {
        'salt': salt,
        'hashed_password': hash_password(password, salt)
    }
    print(f"用户 {username} 注册成功！")


def login():
    username = input("用户名：")
    password = input("密码：")

    if username not in users:
        print("用户不存在")
        return False

    # 获取存储的盐和哈希密码
    user_data = users[username]
    stored_hash = user_data['hashed_password']
    salt = user_data['salt']

    # 对输入密码进行相同哈希计算
    input_hash = hash_password(password, salt)

    # 安全比较哈希值（防止时序攻击）
    if hmac.compare_digest(input_hash, stored_hash):
        print("登录成功！")
        return True
    else:
        print("密码错误")
        return False


# 测试示例
if __name__ == "__main__":
    print("=== 用户注册 ===")
    register_user()

    print("\n=== 登录验证 ===")
    while not login():
        print("请重试...")

=== 用户注册 ===
用户 speed 注册成功！

=== 登录验证 ===
登录成功！


# Python的海象运算符及应用场景示例
:=(形似海象的眼睛和獠牙而得名) Python3.8引入的新特性

In [ ]:
'''场景1: 条件表达式，将赋值和条件判断合并，减少代码行数'''
# 传统写法
user_input = input("请输入数字：")
if int(user_input) > 100:
    print(f"输入的值 {user_input} 过大")

# 使用海象运算符
if (num := int(input("请输入数字："))) > 100:
    print(f"输入的值 {num} 过大")

'''场景2: 循环控制'''
# 密码验证: 在while循环中直接获取用户输入并判断
# 传统写法
while True:
    password = input("设置密码：")
    if len(password) >= 8:
        break
    print("密码需至少8位")

# 使用海象运算符
while len(password := input("设置密码：")) < 8:
    print("密码需至少8位")
# 文件读取: 逐行读取文件内容避免冗余代码
# 传统写法
with open("data.txt") as f:
    line = f.readline()
    while line:
        print(line.strip())
        line = f.readline()

# 使用海象运算符
with open("data.txt") as f:
    while line := f.readline():
        print(line.strip())

'''场景3: 列表推导式优化性能，避免重复计算函数结果'''
import random

# 传统写法（重复计算 random.randint）
numbers = [random.randint(1, 100) for _ in range(10)]
filtered_1 = [x for x in numbers if x % 2 == 0]

# 使用海象运算符（避免重复计算）
filtered_2 = [y for x in numbers if (y := x) % 2 == 0]

# 更复杂场景：计算平方后筛选
squares = [squared for num in numbers if (squared := num ** 2) > 50]

'''场景4: 复杂条件判定'''
# 检查多个条件是否满足（传统写法）
data = {"value": 42}
if "value" in data:
    value = data["value"]
    if value > 0:
        print(f"正数: {value}")

# 使用海象运算符
if (value := data.get("value")) is not None and value > 0:
    print(f"正数: {value}")

# 正则表达式匹配
import re

text = "Price: $99.99"
if match := re.search(r"\$(\d+\.\d+)", text):
    print(f"找到价格: {match.group(1)}")